## Use opence-v1.4.1

In [1]:
#  # mount drive
# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)
# root_dir = "/content/gdrive/MyDrive"
# # change the OS to use your project folder as the working directory
# notebooks = "/post_ocr_repository/post_ocr_correction/notebooks/en"
# import os
# os.chdir(root_dir + notebooks)

In [10]:
# # HAL
# file_dir = '/home/jnaiman/wwt_image_extraction/alignments/'
# output_folder = '/home/jnaiman/data/morgan/'
# # utils from local
# from sys import path
# path.append('/home/jnaiman/libraries/')
# from utils_ocr_mini import align_texts_fast

# local
file_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/alignments/'
output_folder = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/'
from sys import path
path.append('../scienceDigitization/text_mining_ocr_and_pdf/ocr_spell_check/')
from utils import align_texts_fast


# train_file = 'train_masked_n500000_20230503.csv'
# val_file = 'val_masked_n10000_20230503.csv'
# test_file = 'test_masked_n10000_20230503.csv'
# ender = '_large'

# train_file = 'train_masked_n200000_20230502.csv'
# val_file = 'val_masked_n5000_20230502.csv'
# test_file = 'test_masked_n5000_20230502.csv'
# ender = '_regular'

train_file = 'train_masked_n100000_20230510.csv'
val_file = 'val_masked_n5000_20230510.csv'
test_file = 'test_masked_n5000_20230510.csv'
ender = '_small'


window_length = 100 # for subsetting the data

In [7]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd

#from sys import path
#path.append('/home/jnaiman/.local')

from nltk.lm import Vocabulary
#import sys
#sys.path.append("../../lib")
#from metrics import levenshtein
import pickle
from glob import glob

from Levenshtein import distance
import re
from tqdm.auto import tqdm
import Levenshtein

import time
from multiprocessing import Pool


# from importlib import reload
# import utils_ocr_mini
# reload(utils_ocr_mini)
# from utils_ocr_mini import align_texts_fast

In [8]:
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]

def levenshtein(reference, hypothesis, progress_bar = False):
    assert len(reference) == len(hypothesis)
    text = zip(reference, hypothesis)
    if progress_bar:
        text = tqdm(text, total = len(reference))
    d = [distance(r, h) for r, h in text]
    output = pd.DataFrame({"reference":reference, "hypothesis":hypothesis})\
    .assign(distance = lambda df: d)\
    .assign(
        cer = lambda df: df.apply(
            lambda r: 100 * r["distance"] / max(len(r["reference"]), 1), 
            axis = 1
        )
    )
    return output

In [11]:
data1 = pd.read_csv(file_dir + train_file)
data2 = pd.read_csv(file_dir + val_file)
data3 = pd.read_csv(file_dir + test_file)

data = pd.concat([data1,data2,data3],ignore_index=True)

data = data.rename(columns={"sentences source": "ocr_aligned", 
                        "sentences target": "gs_aligned"})

In [12]:
#data.loc[ (pd.isnull(data['ocr_aligned'])) | (pd.isnull(data['gs_aligned'])) ]
mask = (pd.isnull(data['ocr_aligned'])) | (pd.isnull(data['gs_aligned']))

In [13]:
len(data[mask])

47

In [14]:
data.head()

,aligned sentences source,aligned sentences target,ocr_aligned,gs_aligned,aligned sentences source types,aligned sentences target types,sentences source types,sentences target types
0,. Early two-dimensional observations of filame...,@ Early two-dimensional observations of filame...,. Early two-dimensional observations of filame...,Early two-dimensional observations of filamen...,W WWWWW WWWWWWWWWWWWWWW WWWWWWWWWWWW WW WWWWWW...,@ WWWWW WWWWWWWWWWWWWWW WWWWWWWWWWWW WW WWWWWW...,W WWWWW WWWWWWWWWWWWWWW WWWWWWWWWWWW WW WWWWWW...,WWWWW WWWWWWWWWWWWWWW WWWWWWWWWWWW WW WWWWWWW...
1,However. what is remarkable here is that whil...,"However, what is remarkable here is that whil...",However. what is remarkable here is that whil...,"However, what is remarkable here is that whil...",WWWWWWWW WWWW WW WWWWWWWWWW WWWW WW WWWW WWWW...,WWWWWWWW WWWW WW WWWWWWWWWW WWWW WW WWWW WWWW...,WWWWWWWW WWWW WW WWWWWWWWWW WWWW WW WWWW WWWW...,WWWWWWWW WWWW WW WWWWWWWWWW WWWW WW WWWW WWWW...
2,"Therefore, the non-detection of the GRXE in t...","Therefore, the non-detection of the GRXE in t...","Therefore, the non-detection of the GRXE in t...","Therefore, the non-detection of the GRXE in t...",WWWWWWWWWW WWW WWWWWWWWWWWWW WW WWW WWWW WW W...,WWWWWWWWWW WWW WWWWWWWWWWWWW WW WWW WWWW WW W...,WWWWWWWWWW WWW WWWWWWWWWWWWW WW WWW WWWW WW W...,WWWWWWWWWW WWW WWWWWWWWWWWWW WW WWW WWWW WW W...
3,We- vote tha^ the age estimate by Weinberger ...,We@ note that the age estimate by Weinberger ...,We- vote tha the age estimate by Weinberger e...,We note that the age estimate by Weinberger e...,WWW WWWW WWW^ WWW WWW WWWWWWWW WW WWWWWWWWWW ...,WW@ WWWW WWWW WWW WWW WWWWWWWW WW WWWWWWWWWW ...,WWW WWWW WWW WWW WWW WWWWWWWW WW WWWWWWWWWW W...,WW WWWW WWWW WWW WWW WWWWWWWW WW WWWWWWWWWW W...
4,lem When the accretion rate is slightly lower...,1cm When the accretion rate is slightly lower...,lem When the accretion rate is slightly lower...,1cm When the accretion rate is slightly lower...,WWW WWWW WWW WWWWWWWWW WWWW WW WWWWWWWW WWWWW...,WWW WWWW WWW WWWWWWWWW WWWW WW WWWWWWWW WWWWW...,WWW WWWW WWW WWWWWWWWW WWWW WW WWWWWWWW WWWWW...,WWW WWWW WWW WWWWWWWWW WWWW WW WWWWWWWW WWWWW...


In [15]:
data[~mask].applymap(len).describe()

,aligned sentences source,aligned sentences target,ocr_aligned,gs_aligned,aligned sentences source types,aligned sentences target types,sentences source types,sentences target types
count,109953.000000,109953.000000,109953.000000,109953.000000,109953.000000,109953.000000,109953.000000,109953.000000
mean,154.031941,154.031941,142.740280,152.075305,154.031941,154.031941,142.772403,152.075396
std,104.691838,104.691838,84.971713,103.486459,104.691838,104.691838,84.990467,103.486864
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,93.000000,93.000000,86.000000,92.000000,93.000000,93.000000,86.000000,92.000000
50%,140.000000,140.000000,132.000000,139.000000,140.000000,140.000000,132.000000,139.000000
75%,199.000000,199.000000,186.000000,196.000000,199.000000,199.000000,186.000000,196.000000
max,10107.000000,10107.000000,2053.000000,10107.000000,10107.000000,10107.000000,2054.000000,10107.000000


In [16]:
levenshtein(reference = data[~mask].gs_aligned, 
            hypothesis = data[~mask].ocr_aligned).cer.describe()

count    109953.000000
mean         13.116388
std          25.279160
min           0.000000
25%           2.464789
50%           7.738095
75%          17.557252
max        2900.000000
Name: cer, dtype: float64

Create vocabulary:

In [ ]:
rerun_vocab = False

if rerun_vocab:
    start_time = time.time()
    vocabulary = Vocabulary(data[~mask].ocr_aligned.sum() + data[~mask].gs_aligned.sum())
    print(len(vocabulary))
    with open(output_folder+"/"+"data/vocabulary_new_pages_new"+ender+".pkl", "wb") as file:
        pickle.dump(vocabulary, file)
    end_time = time.time()
    print('time delta =', end_time-start_time, 'seconds, or', 
          (end_time-start_time)/60., 'minutes, or',
         (end_time-start_time)/60./60., 'hours')
    # last run:
    #time delta = 11285.80526137352 seconds, 188.09675435622532 minutes, ~3.3 hours on HAL

## Create windowed data

In [11]:
# source_dataframe = pd.DataFrame({"ocr_aligned":[],"gs_aligned":[]})

# #for file_name in files:
# #for file_index in range(len(files)):# can change length of files ex: files/10 or 1000 to find out if there is less time
# import random
# #r = list(range(4773))
# r = list(range(40))
# random.shuffle(r)
# pdf_ocr = []
# for file_index in r: 
#     file_name = files[file_index]
#     if file_index % 200 == 0:
#         print(str(file_index) + " " + file_name)
#     for key in realign_pages:
#         if 'PDF' in realign_pages[key] and 'OCR' in realign_pages[key]:
#             pdf_ocr.append({'ocr_aligned': realign_pages[key]['OCR'],'gs_aligned': realign_pages[key]['PDF']})

#     source_dataframe = pd.DataFrame(pdf_ocr)

Read in only the training data and do stuff with that:

In [ ]:
train = pd.read_csv(file_dir + train_file)

# train = train.rename(columns={"sentences source": "ocr_aligned", 
#                         "sentences target": "gs_aligned"})

#mask = (pd.isnull(train['ocr_aligned'])) | (pd.isnull(train['gs_aligned']))
mask = (pd.isnull(train['sentences source'])) | (pd.isnull(train['sentences target']))

train = train[~mask]

# now align
gs_aligned = []; ocr_aligned = []
for o,p in zip(train['sentences source'].values, train['sentences target'].values):
    eops = Levenshtein.editops(o, p)
    ocr_text_aligned, pdf_text_aligned = align_texts_fast(o,p,eops)
    gs_aligned.append(pdf_text_aligned)
    # futze with ocr aligned so it matches data input
    ocr_text_aligned = ocr_text_aligned.replace('^', '@')
    ocr_aligned.append(ocr_text_aligned)
    
train['gs_aligned'] = gs_aligned
train['ocr_aligned'] = ocr_aligned

In [ ]:
p = Pool(4)

In [ ]:
#train = data.drop(dev.index)
train.to_pickle(output_folder+"/data/train_new_pages"+ender+".pkl")
train.shape

In [ ]:
# df = train#.head(100)
# train_aligned = list(p.imap_unordered(create_windows, 
#                                       tqdm(zip(df.ocr_aligned, 
#                                                df.gs_aligned, 
#                                                [window_length for x in df.ocr_aligned]), 
#                                            total = len(df.ocr_aligned)),
#                                       chunksize = 128))
# s = []
# for r in tqdm(train_aligned):
#     s.extend(r)
# train_aligned = pd.DataFrame(s, columns = ["source", "target"])
# print(train_aligned.shape)
# train_aligned.head()
start_time = time.time()
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(train.ocr_aligned, 
                                               train.gs_aligned, 
                                               [window_length for x in train.ocr_aligned]), 
                                           total = len(train.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()
end_time = time.time()
print('total time =', end_time-start_time, 'seconds, or', (end_time-start_time)/60., 'minutes')

In [ ]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

Same thing for validation set:

In [ ]:
valid = pd.read_csv(file_dir + val_file)

mask = (pd.isnull(valid['sentences source'])) | (pd.isnull(valid['sentences target']))

valid = valid[~mask]

# now align
gs_aligned = []; ocr_aligned = []
for o,p in zip(valid['sentences source'].values, valid['sentences target'].values):
    eops = Levenshtein.editops(o, p)
    ocr_text_aligned, pdf_text_aligned = align_texts_fast(o,p,eops)
    gs_aligned.append(pdf_text_aligned)
    # futze with ocr aligned so it matches data input
    ocr_text_aligned = ocr_text_aligned.replace('^', '@')
    ocr_aligned.append(ocr_text_aligned)
    
valid['gs_aligned'] = gs_aligned
valid['ocr_aligned'] = ocr_aligned

In [ ]:
dev_aligned = valid.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

In [ ]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

Finally save:

In [ ]:
train_aligned.to_pickle(output_folder+"/data/train_aligned_new_pages"+ender+".pkl")
dev_aligned.to_pickle(output_folder+"/data/dev_aligned_new_pages"+ender+".pkl")

# --- orig stuff ---

In [2]:
new_data_dir = '/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/'

In [3]:
output_folder = '/home/mcosi153/post_ocr_correction/data/en'

In [4]:
new_data_files = sorted(os.listdir(new_data_dir))
len(new_data_files)

7892

In [6]:
new_data_files = glob(new_data_dir+"*.pickle")

In [7]:
new_data_files[:3]

['/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/1005_1005.2297d_14607_psfilefixedRTM_hocr.pickle',
 '/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/9811_astro-ph9811382d_buzzoni_psfilefixedRTM_hocr.pickle',
 '/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/0908_0908.4092d_Jenkins09b_psfilefixedRTM_hocr.pickle']

In [8]:
len(new_data_files)

7891

In [9]:
file0 = 1
file_name = new_data_files[file0]

In [10]:

!pip install lxml


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [11]:
# with open(file_name,'rb') as f:
#     ffout,pdf_text_pages,ocr_text_pages,alignment_pages,pdf_sents_aligned_pages,ocr_sents_aligned_pages,pdf_sents_pages,ocr_sents_pages = pickle.load(f)

In [12]:
with open(file_name,'rb') as f:
    realign_pages = pickle.load(f)


In [13]:
len(realign_pages)

9

In [14]:
#len(pdf_sents_aligned_pages)

In [15]:
#files = sorted(os.listdir(folder))
#len(files)
files = sorted(os.listdir(new_data_dir))
len(files)
files[:3]

['.ipynb_checkpoints',
 '0001_astro-ph0001013d_letter_psfilefixedRTM_hocr - Copy.pickle',
 '0001_astro-ph0001013d_letter_psfilefixedRTM_hocr.pickle']

In [16]:
import glob

files = glob.glob(new_data_dir + '/**/*.pickle', recursive=True)
len(files)

7891

In [17]:
from multiprocessing import Pool

# def extract(name):
   # with open(name) as file: #this is used to open and read text files
        #return file.readlines()


def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
    
p = Pool(4)
    
# #data = list(p.imap_unordered(extract, tqdm(files), chunksize = 128,))
# #len(data)
# #data = extract('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle')

source_dataframe = pd.DataFrame({"ocr_aligned":[],"gs_aligned":[]})

#for file_name in files:
#for file_index in range(len(files)):# can change length of files ex: files/10 or 1000 to find out if there is less time
import random
#r = list(range(4773))
r = list(range(40))
random.shuffle(r)
pdf_ocr = []
for file_index in r: 
    file_name = files[file_index]
    if file_index % 200 == 0:
        print(str(file_index) + " " + file_name)
    for key in realign_pages:
        if 'PDF' in realign_pages[key] and 'OCR' in realign_pages[key]:
            pdf_ocr.append({'ocr_aligned': realign_pages[key]['OCR'],'gs_aligned': realign_pages[key]['PDF']})

    source_dataframe = pd.DataFrame(pdf_ocr)
    #fh = open(str(file_name), 'rb')
    #ffout,pdf_text_pages, ocr_text_pages, alignment_pages, pdf_sents_aligned_pages, ocr_sents_aligned_pages, pdf_sents_pages, ocr_sents_pages = pickle.load(fh)
    ##print(len(pdf_sents_aligned_pages.keys()))
    ##import sys; sys.exit()
    #for key in pdf_sents_aligned_pages.keys():
      ##for ocr_sents, pdf_sents in zip(ocr_sents_aligned_pages[key], pdf_sents_aligned_pages[key]):
        #if ocr_sents_aligned_pages[key] != "" and pdf_sents_aligned_pages[key] != "":
        ##print("not empty")
        ##print(key)
            #new_dataframe = pd.DataFrame({"ocr_aligned":ocr_sents_aligned_pages[key], "gs_aligned":pdf_sents_aligned_pages[key]})
            #source_dataframe = pd.concat([source_dataframe, new_dataframe], ignore_index = True, sort = False)
#         else:
#             #print("empty string")
#             print(key)

# #use source_dataframe add code here to remove str from list
# #empty_list = []
#print(source_dataframe.shape)

# # for index in range(len(source_dataframe.ocr_aligned)-1):
# #   if type(source_dataframe.ocr_aligned[index]) != list:
# #     source_dataframe.ocr_aligned.pop(index)
# #     source_dataframe.gs_aligned.pop(index)

# #print(source_dataframe.shape)

0 /home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/1005_1005.2297d_14607_psfilefixedRTM_hocr.pickle


In [18]:
source_dataframe.head()

,ocr_aligned,gs_aligned
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...
2,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
3,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
4,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...


In [19]:
# for index_number in range(len(source_dataframe.gs_aligned)):
#     if index_number % 5000 == 0:
#       #print(source_dataframe.ocr_aligned[index_number])
#       #print(source_dataframe.gs_aligned[index_number])

In [54]:
#source_dataframe.to_csv("full_source_dataframe_to_split.csv")

In [19]:
# the rest of the code is using "data" for dataframe
data = source_dataframe 
#data = pd.read_csv("full_train_data_672699.csv" )

In [20]:
#check how many rows in data
print(data.shape)

(120, 2)


In [21]:
data.head()

,ocr_aligned,gs_aligned
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...
2,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
3,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
4,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...


In [38]:
#data = data.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

In [25]:
#this is what i added to make a list with one pickle file
#lst = pd.read_pickle('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle')
#with open('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle','rb') as f:
# ffout,pdf_text_pages,ocr_text_pages,alignment_pages,pdf_sents_aligned_pages,ocr_sents_aligned_pages,pdf_sents_pages,ocr_sents_pages = pickle.load(f)

In [26]:
#df1 = pd.read_pickle('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle')

In [27]:
#print(df1.shape)

In [28]:

# data = pd.DataFrame(data, 
#                     columns = ["ocr_to_input", 
#                                "ocr_aligned", 
#                                "gs_aligned"])\
# .assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
#         ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
#         gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

# print(data.shape)
# data.head()



In [29]:
#data.to_csv("./github_training_data.csv")

In [22]:
data.applymap(len).describe()

,ocr_aligned,gs_aligned
count,120.000000,120.000000
mean,77.333333,77.333333
std,0.946762,0.946762
min,76.000000,76.000000
25%,76.000000,76.000000
50%,78.000000,78.000000
75%,78.000000,78.000000
max,78.000000,78.000000


In [23]:
!pip install Levenshtein
import pandas as pd
from Levenshtein import distance
import re
from tqdm.auto import tqdm
       
def levenshtein(reference, hypothesis, progress_bar = False):
    assert len(reference) == len(hypothesis)
    text = zip(reference, hypothesis)
    if progress_bar:
        text = tqdm(text, total = len(reference))
    d = [distance(r, h) for r, h in text]
    output = pd.DataFrame({"reference":reference, "hypothesis":hypothesis})\
    .assign(distance = lambda df: d)\
    .assign(
        cer = lambda df: df.apply(
            lambda r: 100 * r["distance"] / max(len(r["reference"]), 1), 
            axis = 1
        )
    )
    return output

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [41]:
# levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
#             hypothesis = data.ocr_to_input).cer.describe()

In [24]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    120.000000
mean       6.848853
std        4.233466
min        1.315789
25%        1.315789
50%        7.692308
75%       11.538462
max       11.538462
Name: cer, dtype: float64

In [105]:
vocabulary = Vocabulary(data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))
with open(output_folder+"/"+"data/vocabulary_new_pages1.pkl", "wb") as file:
    pickle.dump(vocabulary, file)

34


In [35]:
# data.gs_aligned[51173]

In [36]:
#output_folder = Path("../../data/en")

In [37]:
# data.ocr_aligned.pop(51173)
# data.gs_aligned.pop(51173)

In [38]:
# type(data.ocr_aligned[51172])

In [39]:
# vocab1 = []
# vocab2 = []

# for index in range(len(data.ocr_aligned)-1):
#   if type(data.ocr_aligned[index]) != type(vocab1):
#     print(data.ocr_aligned[index])

#   else:
#       vocab1 = vocab1 + data.ocr_aligned[index]
#     #source_dataframe.ocr_aligned.pop()
#     #source_dataframe.gs_aligned.pop()

# for index in range(len(data.gs_aligned)-1):
#   if type(data.gs_aligned[index]) != type(vocab2):
#     print(data.gs_aligned[index])
#   else:
#       vocab2 = vocab2 + data.gs_aligned[index]
#     #source_dataframe.ocr_aligned.pop()

# vocab = vocab1 + vocab2




In [40]:
# data.gs_aligned.sum() 

In [41]:
# for d in data.ocr_aligned.values:
#   if type(d) != list:
#     print(d, "here")

In [11]:
data.gs_aligned.values [- 1]

'@e confront the two models from Table @n this wavelength interval with each other. '

In [54]:
ocr = data.ocr_aligned.sum()
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:38


In [55]:
gs = data.gs_aligned.sum()
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:39


In [56]:
#vocabulary = Vocabulary(data.ocr_aligned.sum() + data.gs_aligned.sum())
vocab = ocr + gs
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:40


In [57]:
vocabulary = Vocabulary(ocr + gs)
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:41


In [58]:
# vocabulary = Vocabulary(vocab)
print(len(vocabulary))
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)


34
Current Time = 15:55:42


In [59]:
with open(output_folder+"/data/vocabulary_new_pages.pkl", "wb") as file:
    pickle.dump(vocabulary, file)
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:46


In [25]:
dev = data.sample(n = 5, random_state = 1)
dev.to_pickle(output_folder+"/data/dev_new_pages.pkl")
dev.shape

(5, 2)

In [26]:
train = data.drop(dev.index)
train.to_pickle(output_folder+"/data/train_new_pages.pkl")
train.shape

(115, 2)

In [27]:
train.applymap(len).describe()

,ocr_aligned,gs_aligned
count,115.000000,115.000000
mean,77.356522,77.356522
std,0.938376,0.938376
min,76.000000,76.000000
25%,76.000000,76.000000
50%,78.000000,78.000000
75%,78.000000,78.000000
max,78.000000,78.000000


In [28]:
dev.applymap(len).describe()

,ocr_aligned,gs_aligned
count,5.000000,5.000000
mean,76.800000,76.800000
std,1.095445,1.095445
min,76.000000,76.000000
25%,76.000000,76.000000
50%,76.000000,76.000000
75%,78.000000,78.000000
max,78.000000,78.000000


In [29]:
# levenshtein(reference = dev.gs_aligned.str.replace("@", ""), 
#              hypothesis = dev.ocr_to_input).cer.describe()

In [30]:
levenshtein(reference = dev.gs_aligned, 
            hypothesis = dev.ocr_aligned).cer.describe()

count    5.000000
mean     3.866397
std      3.492563
min      1.315789
25%      1.315789
50%      1.315789
75%      7.692308
max      7.692308
Name: cer, dtype: float64

In [31]:
window_length = 100

In [32]:
from multiprocessing import Pool

# def extract(name):
   # with open(name) as file: #this is used to open and read text files
        #return file.readlines()


def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
p = Pool(4)

In [33]:
df = train#.head(100)
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(df.ocr_aligned, 
                                               df.gs_aligned, 
                                               [window_length for x in df.ocr_aligned]), 
                                           total = len(df.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

(9011, 2)


,source,target
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,Pollax population svuthesis has extensively be...,tellar population synthesis has extensively be...
2,ollax population svuthesis has extensively bee...,ellar population synthesis has extensively bee...
3,llax population svuthesis has extensively been...,llar population synthesis has extensively been...
4,lax population svuthesis has extensively been ...,lar population synthesis has extensively been ...


In [34]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,Pollax population svuthesis has extensively be...,tellar population synthesis has extensively be...
2,ollax population svuthesis has extensively bee...,ellar population synthesis has extensively bee...
3,llax population svuthesis has extensively been...,llar population synthesis has extensively been...
4,lax population svuthesis has extensively been ...,lar population synthesis has extensively been ...


In [35]:
dev_aligned = dev.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(389, 2)


,source,target
0,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
1,s a general feature. SSP post-MS evolution is ...,"s a general feature, SSP post-MS evolution is ..."
2,a general feature. SSP post-MS evolution is r...,"a general feature, SSP post-MS evolution is r..."
3,a general feature. SSP post-MS evolution is re...,"a general feature, SSP post-MS evolution is re..."
4,general feature. SSP post-MS evolution is rec...,"general feature, SSP post-MS evolution is rec..."


In [44]:
#trained_sample_df=train_aligned.head(1)

In [45]:
#trained_sample_df.to_csv("trained_sample.csv")

In [36]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
1,s a general feature. SSP post-MS evolution is ...,"s a general feature, SSP post-MS evolution is ..."
2,a general feature. SSP post-MS evolution is r...,"a general feature, SSP post-MS evolution is r..."
3,a general feature. SSP post-MS evolution is re...,"a general feature, SSP post-MS evolution is re..."
4,general feature. SSP post-MS evolution is rec...,"general feature, SSP post-MS evolution is rec..."


In [37]:
train_aligned.to_pickle(output_folder+"/data/train_aligned_new_pages.pkl")
dev_aligned.to_pickle(output_folder+"/data/dev_aligned_new_pages.pkl")

In [14]:
import numpy as np

In [18]:
df = train_aligned

In [19]:
len(df)

91919976

In [16]:
train_aligned_chunks = np.array_split(df, 92)
for i, df in enumerate(train_aligned_chunks):
    if i % 10 == 0 and i != 0:
        print(f"{i} dataframes have been created so far")

10 dataframes have been created so far
20 dataframes have been created so far
30 dataframes have been created so far
40 dataframes have been created so far
50 dataframes have been created so far
60 dataframes have been created so far
70 dataframes have been created so far
80 dataframes have been created so far
90 dataframes have been created so far


In [17]:
for i in range(92):
    train_aligned_chunks[i].to_pickle(output_folder+"/data/train_aligned_new_full"+str(i).zfill(4)+".pkl")

In [21]:
dev_df = dev_aligned
len(dev_df)

635

In [22]:
dev_df.to_pickle(output_folder+"/data/dev_aligned_new_full.pkl")

In [ ]:
train_aligned.to_pickle(output_folder+"/data/train_aligned_new_full.pkl")
dev_aligned.to_pickle(output_folder+"/data/dev_aligned_new_full.pkl")

In [ ]:
train = pd.read_pickle(output_folder+"/data/train_aligned_new_full.pkl")
train.shape

In [4]:
import pandas as pd

In [25]:
import numpy as np

In [26]:
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

In [27]:
df


,A,B,C,D
0,29,94,33,50
1,14,54,75,51
2,35,36,61,66
3,25,9,72,41
4,26,66,9,22
...,...,...,...,...
95,41,27,88,65
96,6,53,39,52
97,85,39,59,92
98,69,51,1,87


In [33]:
df_chunks = np.array_split(df, 10)

In [34]:
df_chunks[0]

,A,B,C,D
0,29,94,33,50
1,14,54,75,51
2,35,36,61,66
3,25,9,72,41
4,26,66,9,22
5,90,64,78,94
6,77,95,99,98
7,34,32,88,43
8,2,80,38,18
9,68,94,10,9


In [35]:
for i in range(10):
    df_chunks[i].to_pickle(output_folder+"/data/train_aligned_new_full"+str(i).zfill(3)+".pkl")

IndexError: list index out of range